In [1]:
import unittest
from bvh import Bvh, BvhNode

In [62]:
import re

bvh_file = "coffee_1Char00.bvh"

def identifier(scanner, token):  return "IDENT", token
def operator(scanner, token):    return "OPERATOR", token
def digit(scanner, token):       return "DIGIT", token
def open_brace(scanner, token):  return "OPEN_BRACE", token
def close_brace(scanner, token): return "CLOSE_BRACE", token

current_token = 0

skeleton = {}
bone_context = []
motion_channels = []
motions = []
bodyParts = [[]]

def new_bone(parent, name):
	bone = { "parent" : parent, "channels" : [], "offsets" : []}
	return bone

def push_bone_context(name):
	global bone_context
	bone_context.append(name)

def get_bone_context():
	global bone_context
	return bone_context[len(bone_context)-1]

def pop_bone_context():
	global bone_context
	bone_context = bone_context[:-1]
	return bone_context[len(bone_context)-1]

reserved      = [ "HIERARCHY", "ROOT", "OFFSET", "CHANNELS", "MOTION" ]
channel_names = [ "Xposition", "Yposition", "Zposition",  "Yrotation", "Xrotation",  "Zrotation" ]

scanner = re.Scanner([
    (r"[a-zA-Z_]\w*", identifier),
    (r"-*[0-9]+(\.[0-9]+)?", digit),
	(r"}", close_brace),
	(r"{", open_brace),
	(r":", None),
    (r"\s+", None),
    ])

def read_offset(bvh, token_index):
	if (bvh[token_index] != ("IDENT", "OFFSET")):
		return None, None
	token_index = token_index + 1
	offsets = [ 0.0 ] * 3
	for i in range(0,3):
		offsets[i] = float(bvh[token_index][1])
		token_index = token_index + 1
	return  offsets, token_index

def read_channels(bvh, token_index):
	if (bvh[token_index] != ("IDENT", "CHANNELS")):
		return None, None
	token_index = token_index + 1
	channel_count = int(bvh[token_index][1])
	token_index = token_index+1
	channels = [ "" ] * channel_count
	for i in range(0, channel_count):
		channels[i] = bvh[token_index][1]
		token_index = token_index+1
	return channels, token_index

def parse_joint(bvh, token_index):
	end_site = False
	joint_id = bvh[token_index][1]
	token_index = token_index + 1
	joint_name = bvh[token_index][1]
	token_index = token_index + 1
	if (joint_id == "End"): # end site
		joint_name = get_bone_context() + "_Nub"
		end_site = True
	joint = new_bone(get_bone_context(), joint_name)
	if bvh[token_index][0] != "OPEN_BRACE":
		print ("Was expecting brace, got ", bvh[token_index])
		return None
	token_index = token_index + 1
	offsets, token_index = read_offset(bvh, token_index)
	joint["offsets"] = offsets
	if (not(end_site)):
		channels, token_index = read_channels(bvh, token_index)
		joint["channels"] = channels
		for channel in channels:
			motion_channels.append((joint_name, channel))
	skeleton[joint_name] = joint
	while (((bvh[token_index][0] == "IDENT") and (bvh[token_index][1] == "JOINT")) or
		  ((bvh[token_index][0] == "IDENT") and (bvh[token_index][1] == "End"))):
		push_bone_context(joint_name)
		token_index = parse_joint(bvh, token_index)
		pop_bone_context()
	if (bvh[token_index][0]) == "CLOSE_BRACE":
		return token_index + 1
	print ("Unexpected token ", bvh[token_index])

def parse_hierarchy(bvh):
	global current_token
	current_token = 0
	if (bvh[current_token] != ("IDENT", "HIERARCHY")):
		return None
	current_token = current_token + 1
	if (bvh[current_token] != ("IDENT", "ROOT")):
		return None
	current_token = current_token + 1
	if (bvh[current_token][0] != "IDENT"):
		return None
	root_name =bvh[current_token][1]
	root_bone = new_bone(None, root_name)
	current_token = current_token + 1
	if bvh[current_token][0] != "OPEN_BRACE":
		return None
	current_token = current_token + 1
	offsets, current_token = read_offset(bvh, current_token)
	channels, current_token = read_channels(bvh, current_token)
	root_bone["offsets"]  = offsets
	root_bone["channels"] = channels
	skeleton[root_name] = root_bone
	push_bone_context(root_name)
	print ("Root ", root_bone)
	while(bvh[current_token][1] == "JOINT"):
		current_token = parse_joint(bvh, current_token)


def parse_motion(bvh):
    global current_token
    if (bvh[current_token][0] != "IDENT"):
        print ("Unexpected text")
        return None
    if (bvh[current_token][1] != "MOTION"):
        print ("No motion section")
        return None
    current_token = current_token + 1
    if (bvh[current_token][1] != "Frames"):
        return None
    current_token = current_token  + 1
    frame_count = int(bvh[current_token][1])
    current_token = current_token  + 1
    if (bvh[current_token][1] != "Frame"):
        return None
    current_token = current_token  + 1
    if (bvh[current_token][1] != "Time"):
        return None
    current_token = current_token  + 1
    frame_rate = float(bvh[current_token][1])
    frame_time = 0.0
    motions = [ () ] * frame_count 
    requiredBodyParts = [] * frame_count
    for i in range(0, frame_count):
        #print ("Parsing frame ", i)
        channel_values = []
        for channel in motion_channels:
            channel_values.append((channel[0], channel[1], float(bvh[current_token][1])))
            current_token = current_token + 1
        motions[i] = (frame_time, channel_values)
        requiredBodyParts.append(channel_values[84:162])
        #print (channel_values[84:162])
        #print(requiredBodyParts[i])
        frame_time = frame_time + frame_rate
    #print(requiredBodyParts)
    return requiredBodyParts
        
#print (motions[i])
#[main_list[x] for x in indexes]

In [ ]:

bvh_file = open("coffee_1Char00.bvh", "r")
bvh = bvh_file.read()
bvh_file.close()
tokens, remainder = scanner.scan(bvh)
parse_hierarchy(tokens)
for (name,bone) in skeleton.items():
    print ("Bone " + name);
    if "channels" in bone:
        print ("Channels ", bone["channels"])
    print ("Offsets ", bone["offsets"])
current_token = current_token + 1
requiredBodyParts = parse_motion(tokens)
#print(requiredBodyParts[1:30])
print("\n\n\n\n\ After reqBodyPart")

sizeList = len(requiredBodyParts)
rightForeArmXPosition = []*sizeList 
rightForeArmYPosition = []*sizeList 
rightForeArmZPosition = []*sizeList 
rightForeArmXRotation = []*sizeList 
rightForeArmYRotation = []*sizeList 
rightForeArmZRotation = []*sizeList 

rightHandXPosition = []*sizeList 
rightHandYPosition = []*sizeList 
rightHandZPosition = []*sizeList 
rightHandXRotation = []*sizeList 
rightHandYRotation = []*sizeList 
rightHandZRotation = []*sizeList 

for i in range(0,sizeList):
    rightForeArmXPosition.append (requiredBodyParts[i][0] )
    rightForeArmYPosition.append (requiredBodyParts[i][1] )
    rightForeArmZPosition.append (requiredBodyParts[i][2] )
    rightForeArmXRotation.append (requiredBodyParts[i][3] )
    rightForeArmYRotation.append (requiredBodyParts[i][4] )
    rightForeArmZRotation.append (requiredBodyParts[i][5] )
    
    rightHandXPosition.append (requiredBodyParts[i][6] )
    rightHandYPosition.append (requiredBodyParts[i][7] )
    rightHandZPosition.append (requiredBodyParts[i][8] )
    rightHandXRotation.append (requiredBodyParts[i][9] )
    rightHandYRotation.append (requiredBodyParts[i][10] )
    rightHandZRotation.append (requiredBodyParts[i][11] )

for i in range (0, sizeList):
    print (i,": ");
    print (rightForeArmXPosition[i][2])
    print (rightForeArmYPosition[i][2])
    print (rightForeArmZPosition[i][2])
    print (rightForeArmXRotation[i][2])
    print (rightForeArmYRotation[i][2])
    print (rightForeArmZRotation[i][2])
    
    print (rightHandXPosition[i][2])
    print (rightHandYPosition[i][2])
    print (rightHandZPosition[i][2])
    print (rightHandXRotation[i][2])
    print (rightHandYRotation[i][2])
    print (rightHandZRotation[i][2])
    
#print (len(rightForeArmXPosition))

#if "channels" in bone.has_key("channels")

Root  {'parent': None, 'channels': ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation'], 'offsets': [0.0, 93.594, 0.0]}
Bone Hips
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [0.0, 93.594, 0.0]
Bone RightUpLeg
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [-9.5, -1.594, 0.0]
Bone RightLeg
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [0.0, -42.0, 0.0]
Bone RightFoot
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [0.0, -42.0, 0.0]
Bone RightFoot_Nub
Channels  []
Offsets  [0.0, -8.0, 17.5]
Bone LeftUpLeg
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [9.5, -1.594, 0.0]
Bone LeftLeg
Channels  ['Xposition', 'Yposition', 'Zposition', 'Yrotation', 'Xrotation', 'Zrotation']
Offsets  [0.0, -42.0, 0.0]
Bone LeftFoo

0.0
-0.0
106 : 
29.102865
-4.062316
0.0
0.0
0.0
-0.0
29.102865
-2.546526
0.0
0.0
0.0
-0.0
107 : 
0.0
-5.321937
-0.0864
0.322863
0.0
-0.0
29.101387
-4.062316
0.0
0.0
0.0
-0.0
108 : 
47.993164
-3.478737
0.532042
0.778358
0.0
-0.0
0.0
-5.321937
-0.0864
0.322863
0.0
-0.0
109 : 
47.979176
-2.110737
0.0
0.0
0.0
-0.0
47.979176
-3.478737
0.532042
0.778358
0.0
-0.0
110 : 
47.97031
-3.722779
0.0
0.0
0.0
-0.0
47.97031
-2.110737
0.0
0.0
0.0
-0.0
111 : 
0.0
-5.365895
-0.093979
1.027705
15.0
-0.0
47.966412
-3.722779
0.0
0.0
0.0
-0.0
112 : 
0.0
-3.31579
0.522947
2.034947
0.0
-0.0
0.0
-5.365895
-0.093979
1.027705
15.0
-0.0
113 : 
9.593056
-2.631411
0.0
0.0
-17.396051
-0.0
0.0
-3.31579
0.522947
2.034947
0.0
-0.0
114 : 
50.948368
-3.787958
0.0
0.0
-23.195374
-0.0
9.592021
-2.631411
0.0
0.0
-17.39653
-0.0
115 : 
-1.859419
-2.560169
0.195537
3.209684
12.67398
-0.0
50.948666
-3.787958
0.0
0.0
-23.198874
-0.0
116 : 
-36.656506
-26.0
0.0
0.0
4.790862
-0.472872
-1.863238
-2.560169
0.195537
3.209684
12.675191


-0.0
0.0
2.560169
0.195537
3.209684
-21.860001
-0.0
271 : 
-66.801414
27.000002
0.0
0.0
0.0
-0.0
0.0
26.0
0.0
0.0
0.0
-0.0
272 : 
0.0
12.799999
0.0
0.0
1e-06
-1e-06
-66.801414
27.000002
0.0
0.0
0.0
-0.0
273 : 
29.204304
3.2
6.944703
0.0
0.0
-0.0
0.0
12.799999
0.0
0.0
1e-06
-1e-06
274 : 
29.211254
-1.791663
0.0
0.0
0.0
-0.0
29.211254
3.2
6.944703
0.0
0.0
-0.0
275 : 
29.212067
-2.835284
0.0
0.0
0.0
-0.0
29.212067
-1.791663
0.0
0.0
0.0
-0.0
276 : 
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
29.21122
-2.835284
0.0
0.0
0.0
-0.0
277 : 
29.211456
-3.251369
0.482779
-1.236126
0.0
-0.0
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
278 : 
29.210546
-2.456337
0.0
0.0
0.0
-0.0
29.210546
-3.251369
0.482779
-1.236126
0.0
-0.0
279 : 
29.20833
-3.540733
0.0
0.0
0.0
-0.0
29.20833
-2.456337
0.0
0.0
0.0
-0.0
280 : 
0.0
-4.767158
-0.022737
-0.492632
-15.0
-0.0
29.20702
-3.540733
0.0
0.0
0.0
-0.0
281 : 
29.20203
-3.461305
0.553263
-0.132632
0.0
-0.0
0.0
-4.767158
-0.022737
-0.492632
-15.0
-0.0
282 : 
29.20

0.0
9.0
0.0
-3e-06
-0.0
417 : 
0.0
0.0
9.746131
0.0
0.0
-0.0
0.0
0.0
10.417054
0.0
3e-06
0.0
418 : 
-0.0
0.0
10.148685
0.0
0.0
-0.0
0.0
0.0
9.746131
0.0
0.0
-0.0
419 : 
0.0
0.0
9.746131
0.0
3e-06
0.0
-0.0
0.0
10.148685
0.0
0.0
-0.0
420 : 
0.0
0.0
14.347799
0.0
-3e-06
-0.0
0.0
0.0
9.746131
0.0
3e-06
0.0
421 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
0.0
14.347799
0.0
-3e-06
-0.0
422 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
423 : 
0.0
9.5
-1.5942
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
424 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
9.5
-1.5942
0.0
0.0
0.0
425 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
426 : 
0.0
-9.5
-1.5942
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
427 : 
-20.0
0.0
93.594193
0.0
-42.113361
0.0
0.0
-9.5
-1.5942
0.0
0.0
0.0
428 : 
-24.0
1.791663
0.0
0.0
0.0
-0.0
-20.0
0.0
93.594193
0.0
-42.113361
0.0
429 : 
-20.0
2.835284
0.0
0.0
0.0
-0.0
-24.0
1.791663
0.0
0.0
0.0
-0.0
430 : 
0.0
4.259368
-0.022737
-1.121684
8.83
-0.0
-20.0
2.835284
0.0
0.0
0.0
-0.0
431 : 
-20.0
3.251369

-2.55
-0.0
562 : 
0.0
2.631411
0.0
0.0
10.0
-0.0
0.0
3.31579
0.522947
2.034947
0.0
-0.0
563 : 
-30.0
3.787958
0.0
0.0
10.0
-0.0
0.0
2.631411
0.0
0.0
10.0
-0.0
564 : 
0.0
2.560169
0.195537
3.209684
-21.860001
-0.0
-30.0
3.787958
0.0
0.0
10.0
-0.0
565 : 
0.0
26.0
0.0
0.0
0.0
-0.0
0.0
2.560169
0.195537
3.209684
-21.860001
-0.0
566 : 
-66.801414
27.000002
0.0
0.0
0.0
-0.0
0.0
26.0
0.0
0.0
0.0
-0.0
567 : 
0.0
12.799999
0.0
0.0
1e-06
-1e-06
-66.801414
27.000002
0.0
0.0
0.0
-0.0
568 : 
28.166512
3.2
6.944703
0.0
0.0
-0.0
0.0
12.799999
0.0
0.0
1e-06
-1e-06
569 : 
28.159943
-1.791663
0.0
0.0
0.0
-0.0
28.159943
3.2
6.944703
0.0
0.0
-0.0
570 : 
28.151562
-2.835284
0.0
0.0
0.0
-0.0
28.151562
-1.791663
0.0
0.0
0.0
-0.0
571 : 
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
28.140669
-2.835284
0.0
0.0
0.0
-0.0
572 : 
28.140604
-3.251369
0.482779
-1.236126
0.0
-0.0
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
573 : 
28.140314
-2.456337
0.0
0.0
0.0
-0.0
28.140314
-3.251369
0.482779
-1.236126
0.0
-0.0
574

...

1.791663
0.0
0.0
0.0
-0.0
-20.0
0.0
93.594193
0.0
-42.113361
0.0
4205 : 
-20.0
2.835284
0.0
0.0
0.0
-0.0
-24.0
1.791663
0.0
0.0
0.0
-0.0
4206 : 
0.0
4.259368
-0.022737
-1.121684
8.83
-0.0
-20.0
2.835284
0.0
0.0
0.0
-0.0
4207 : 
-20.0
3.251369
0.482779
-1.236126
0.0
-0.0
0.0
4.259368
-0.022737
-1.121684
8.83
-0.0
4208 : 
-24.0
2.456337
0.0
0.0
0.0
-0.0
-20.0
3.251369
0.482779
-1.236126
0.0
-0.0
4209 : 
-20.0
3.540733
0.0
0.0
0.0
-0.0
-24.0
2.456337
0.0
0.0
0.0
-0.0
4210 : 
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
-20.0
3.540733
0.0
0.0
0.0
-0.0
4211 : 
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
4212 : 
-24.0
2.546526
0.0
0.0
0.0
-0.0
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
4213 : 
-20.0
4.062316
0.0
0.0
0.0
-0.0
-24.0
2.546526
0.0
0.0
0.0
-0.0
4214 : 
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
-20.0
4.062316
0.0
0.0
0.0
-0.0
4215 : 
-20.0
3.478737
0.532042
0.778358
0.0
-0.0
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
4216 : 
-24.0
2.110737
0.0
0.0
0.

-0.0
130.503693
3.2
6.944703
0.0
0.0
-0.0
4346 : 
130.500381
-2.835284
0.0
0.0
0.0
-0.0
130.500381
-1.791663
0.0
0.0
0.0
-0.0
4347 : 
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
130.512268
-2.835284
0.0
0.0
0.0
-0.0
4348 : 
130.505753
-3.251369
0.482779
-1.236126
0.0
-0.0
0.0
-4.259368
-0.022737
-1.121684
-25.0
-0.0
4349 : 
130.519577
-2.456337
0.0
0.0
0.0
-0.0
130.519577
-3.251369
0.482779
-1.236126
0.0
-0.0
4350 : 
130.520294
-3.540733
0.0
0.0
0.0
-0.0
130.520294
-2.456337
0.0
0.0
0.0
-0.0
4351 : 
0.0
-4.767158
-0.022737
-0.492632
-15.0
-0.0
130.527206
-3.540733
0.0
0.0
0.0
-0.0
4352 : 
130.533813
-3.461305
0.553263
-0.132632
0.0
-0.0
0.0
-4.767158
-0.022737
-0.492632
-15.0
-0.0
4353 : 
130.541
-2.546526
0.0
0.0
0.0
-0.0
130.541
-3.461305
0.553263
-0.132632
0.0
-0.0
4354 : 
130.574875
-4.062316
0.0
0.0
0.0
-0.0
130.574875
-2.546526
0.0
0.0
0.0
-0.0
4355 : 
0.0
-5.321937
-0.0864
0.322863
0.0
-0.0
130.603333
-4.062316
0.0
0.0
0.0
-0.0
4356 : 
117.635513
-3.478737
0.532042
0.778358
0.0

0.195537
3.209684
8.387792
-0.0
4483 : 
-0.562883
-27.000002
0.0
0.0
27.395142
55.580544
-6.838142
-26.0
0.0
0.0
12.084378
2.769085
4484 : 
0.0
-12.799999
0.0
0.0
3.347397
-17.156187
-0.522461
-27.000002
0.0
0.0
27.4275
55.641773
4485 : 
0.0
-3.2
6.944703
0.0
0.0
-0.0
0.0
-12.799999
0.0
0.0
3.304006
-17.223097
4486 : 
-0.0
0.0
9.0
0.0
-3e-06
-0.0
0.0
-3.2
6.944703
0.0
0.0
-0.0
4487 : 
0.0
0.0
10.417054
0.0
3e-06
0.0
-0.0
0.0
9.0
0.0
-3e-06
-0.0
4488 : 
0.0
0.0
9.746131
0.0
0.0
-0.0
0.0
0.0
10.417054
0.0
3e-06
0.0
4489 : 
-0.0
0.0
10.148685
0.0
0.0
-0.0
0.0
0.0
9.746131
0.0
0.0
-0.0
4490 : 
0.0
0.0
9.746131
0.0
3e-06
0.0
-0.0
0.0
10.148685
0.0
0.0
-0.0
4491 : 
0.0
0.0
14.347799
0.0
-3e-06
-0.0
0.0
0.0
9.746131
0.0
3e-06
0.0
4492 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
0.0
14.347799
0.0
-3e-06
-0.0
4493 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
4494 : 
0.0
9.5
-1.5942
0.0
0.0
0.0
0.0
0.0
-42.0
0.0
0.0
0.0
4495 : 
0.0
0.0
-42.0
0.0
0.0
0.0
0.0
9.5
-1.5942
0.0
0.0
0.0
4496 : 
0.0
0.0

-0.0
4621 : 
-24.0
2.456337
0.0
0.0
0.0
-0.0
-20.0
3.251369
0.482779
-1.236126
0.0
-0.0
4622 : 
-20.0
3.540733
0.0
0.0
0.0
-0.0
-24.0
2.456337
0.0
0.0
0.0
-0.0
4623 : 
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
-20.0
3.540733
0.0
0.0
0.0
-0.0
4624 : 
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
4625 : 
-24.0
2.546526
0.0
0.0
0.0
-0.0
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
4626 : 
-20.0
4.062316
0.0
0.0
0.0
-0.0
-24.0
2.546526
0.0
0.0
0.0
-0.0
4627 : 
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
-20.0
4.062316
0.0
0.0
0.0
-0.0
4628 : 
-20.0
3.478737
0.532042
0.778358
0.0
-0.0
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
4629 : 
-24.0
2.110737
0.0
0.0
0.0
-0.0
-20.0
3.478737
0.532042
0.778358
0.0
-0.0
4630 : 
-20.0
3.722779
0.0
0.0
0.0
-0.0
-24.0
2.110737
0.0
0.0
0.0
-0.0
4631 : 
0.0
5.365895
-0.093979
1.027705
-2.55
-0.0
-20.0
3.722779
0.0
0.0
0.0
-0.0
4632 : 
0.0
3.31579
0.522947
2.034947
0.0
-0.0
0.0
5.365895
-0.093979
1.027705
-2.55
-0.0
4633 : 
0.0
2.6

-0.0
4740 : 
-20.0
3.540733
0.0
0.0
0.0
-0.0
-24.0
2.456337
0.0
0.0
0.0
-0.0
4741 : 
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
-20.0
3.540733
0.0
0.0
0.0
-0.0
4742 : 
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
0.0
4.767158
-0.022737
-0.492632
3.0
-0.0
4743 : 
-24.0
2.546526
0.0
0.0
0.0
-0.0
-20.0
3.461305
0.553263
-0.132632
0.0
-0.0
4744 : 
-20.0
4.062316
0.0
0.0
0.0
-0.0
-24.0
2.546526
0.0
0.0
0.0
-0.0
4745 : 
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
-20.0
4.062316
0.0
0.0
0.0
-0.0
4746 : 
-20.0
3.478737
0.532042
0.778358
0.0
-0.0
0.0
5.321937
-0.0864
0.322863
0.0
-0.0
4747 : 
-24.0
2.110737
0.0
0.0
0.0
-0.0
-20.0
3.478737
0.532042
0.778358
0.0
-0.0
4748 : 
-20.0
3.722779
0.0
0.0
0.0
-0.0
-24.0
2.110737
0.0
0.0
0.0
-0.0
4749 : 
0.0
5.365895
-0.093979
1.027705
-2.55
-0.0
-20.0
3.722779
0.0
0.0
0.0
-0.0
4750 : 
0.0
3.31579
0.522947
2.034947
0.0
-0.0
0.0
5.365895
-0.093979
1.027705
-2.55
-0.0
4751 : 
0.0
2.631411
0.0
0.0
10.0
-0.0
0.0
3.31579
0.522947
2.034947
0.0
-0.0
4752 : 
-30.0
3.7879

In [20]:
hanja = [i for i in range(1,100)];
def abc():
    hanja = [i for i in range(1,100)];
    
banja = hanja[5:45]
print (banja);


[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]
